# Google Cloud - Retail Search API Quick Start

With Retail Search, retailers can implement Google-quality search that is customizable and built upon Google's understanding of user intent and context.

### Install the libraries used in this tutorial

In [ ]:
pip install --upgrade --user google-cloud-retail

In [ ]:
pip install --upgrade --user protobuf

In [ ]:
pip install --upgrade google-auth

### Install the libraries used in this tutorial & Authentification credentials

In this block we import librairy retail_v2 and configure project

In [130]:
from google.cloud.retail_v2.services.search_service import SearchServiceClient
from google.cloud.retail_v2.types.search_service import SearchRequest
from google.cloud.retail_v2 import Product
import google.cloud.retail_v2.services.search_service
import google.auth
import json
from typing import List, Dict, Sequence, Any

#Global Settings
project_id='pod-fr-retail'
project_number='486742359899'
project,credentials = google.auth.default(quota_project_id=project_id)


### Get started with a simple query: Search Client, Query Specs, Query

Let's start very simple. We instanciate a client (SearchServiceClient) and set the required field to have some product results from Retail Search 

In [137]:
#Client
client = SearchServiceClient()

#Query Specs
search_request = SearchRequest()
search_request.query = 'shirt'
search_request.visitor_id = "123"
search_request.placement = 'projects/{}/locations/global/catalogs/default_catalog/placements/default_search'.format(project_id)
search_request.branch='projects/{}/locations/global/catalogs/default_catalog/branches/1'.format(project_number)

Let's have a look at filters and faceting capabilities. Facets are filters you usually see on the left or right side of a search results page.
Documentation https://cloud.google.com/retail/docs/reference/rest/v2/FacetSpec
Example: https://www.addsearch.com/wp-content/uploads/2019/10/Screen-Shot-2019-10-14-at-17.28.51.png

In [138]:
#Client
client = SearchServiceClient()

#Query Specs
search_request = SearchRequest()
search_request.query = 'shirt'
search_request.visitor_id = "123"
search_request.placement = 'projects/{}/locations/global/catalogs/default_catalog/placements/default_search'.format(project_id)
search_request.branch='projects/{}/locations/global/catalogs/default_catalog/branches/1'.format(project_number)
search_request.filter = "categories: ANY(\"Men > Active > Active\")"
search_request.facet_specs= [
        { 
            'facet_key': { 
                'key': 'categories' 
                }
                ,'excluded_filter_keys': ["categories"] #won't filter categories' counting for categories facet
            }
            ,{ 'facet_key': {
                'key': 'availability'
                }
             }
        ]
#Query
response = client.search(search_request)
print(response.results)

[id: "18671"
product {
  name: "projects/486742359899/locations/global/catalogs/default_catalog/branches/1/products/18671"
  categories: "Men > Active > Active"
  title: "Adidas Men\'s Climacore Short Sleeve Shirt"
  brands: "adidas"
  attributes {
    key: "estimatedDeliveryDay"
    value {
      numbers: 4
      searchable: false
      indexable: true
    }
  }
  price_info {
    currency_code: "USD"
    price: 25
    original_price: 25
    cost: 10.75
  }
  uri: "https://retail-demo.mystorefront.org/product/18671"
  images {
    uri: "https://storage.googleapis.com/css-retail-images/subcat_active.png"
    height: 200
    width: 146
  }
  color_info {
    colors: "RED"
  }
  sizes: "XXL"
}
, id: "18379"
product {
  name: "projects/486742359899/locations/global/catalogs/default_catalog/branches/1/products/18379"
  categories: "Men > Active > Active"
  title: "Helly Hansen Men\'s Stripe Crew Shirt"
  brands: "Helly Hansen"
  attributes {
    key: "estimatedDeliveryDay"
    value {
    

Here is a way to create intervals for a numeric facet key

In [138]:
search_request.facet_specs = [{
    'facet_key': {
        'key':
            'price',
        'intervals': [{
            'exclusive_maximum': 100.0
        }, {
            'minimum': 0.0,
            'maximum': 2.0
        }, {
            'minimum': 0.0,
            'maximum': 6.0
        }, {
            'minimum': 0.0,
            'maximum': 10.0
        }, {
            'minimum': 2.0,
            'maximum': 6.0
        }, {
            'minimum': 6.0,
            'maximum': 30.0
        }, {
            'exclusive_minimum': 2.0
        }]
    }
}]
response = client.search(search_request)

print((response.facets[0:1]))

[id: "18671"
product {
  name: "projects/486742359899/locations/global/catalogs/default_catalog/branches/1/products/18671"
  categories: "Men > Active > Active"
  title: "Adidas Men\'s Climacore Short Sleeve Shirt"
  brands: "adidas"
  attributes {
    key: "estimatedDeliveryDay"
    value {
      numbers: 4
      searchable: false
      indexable: true
    }
  }
  price_info {
    currency_code: "USD"
    price: 25
    original_price: 25
    cost: 10.75
  }
  uri: "https://retail-demo.mystorefront.org/product/18671"
  images {
    uri: "https://storage.googleapis.com/css-retail-images/subcat_active.png"
    height: 200
    width: 146
  }
  color_info {
    colors: "RED"
  }
  sizes: "XXL"
}
, id: "18379"
product {
  name: "projects/486742359899/locations/global/catalogs/default_catalog/branches/1/products/18379"
  categories: "Men > Active > Active"
  title: "Helly Hansen Men\'s Stripe Crew Shirt"
  brands: "Helly Hansen"
  attributes {
    key: "estimatedDeliveryDay"
    value {
    

We are now going to dive into Local inventories. How to set and query these local inventory.
* Documentation https://cloud.google.com/retail/docs/reference/rest/v2/FacetSpec
* Example: https://www.addsearch.com/wp-content/uploads/2019/10/Screen-Shot-2019-10-14-at-17.28.51.png
##### Let's start with setting a local inventory for product id 16684 and for store789

In [93]:
from google.cloud import retail_v2

# Create a client
client = retail_v2.ProductServiceClient()

# Initialize request argument(s)
request = retail_v2.AddLocalInventoriesRequest(
    product="projects/486742359899/locations/global/catalogs/default_catalog/branches/1/products/16684",
    local_inventories= [ 
        retail_v2.LocalInventory( 
            place_id= 'store789'
            , price_info= retail_v2.PriceInfo(
                currency_code= 'USD',
                price = 1.25,
                original_price=1.26
            ))
    ]
)

# Make the request
operation = client.add_local_inventories(request=request, timeout=200)
import time

print("Waiting for operation to complete...")
while operation.running():
    print(operation.running())
    time.sleep(2.5)
if operation.done():
    print("Done...")

Waiting for operation to complete...
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
Done...


##### We can now retieve a specific price for product 16684 in the store789 in variant_rollup_values key.

In [167]:
#Client
client = SearchServiceClient()

#Query Specs
search_request = SearchRequest()
search_request.query = 'shirt'
search_request.visitor_id = "16684"
search_request.placement = 'projects/{}/locations/global/catalogs/default_catalog/placements/default_search'.format(project_id)
search_request.branch='projects/{}/locations/global/catalogs/default_catalog/branches/1'.format(project_number)
search_request.variant_rollup_keys= ["inventory(store789,price)"] #https://cloud.google.com/retail/docs/reference/rest/v2alpha/projects.locations.catalogs.placements/search#request-body
#Query
response = client.search(search_request)
print(response )

SearchPager<results {
  id: "16684"
  product {
    name: "projects/486742359899/locations/global/catalogs/default_catalog/branches/1/products/16684"
    categories: "Men > Clothing > Tops & Tees"
    title: "Diesel Men\'s Stombol-S Shirt"
    brands: "Diesel"
    attributes {
      key: "estimatedDeliveryDay"
      value {
        numbers: 4
        searchable: false
        indexable: true
      }
    }
    price_info {
      currency_code: "USD"
      price: 98
      original_price: 98
      cost: 56.644
    }
    uri: "https://retail-demo.mystorefront.org/product/16684"
    images {
      uri: "https://storage.googleapis.com/css-retail-images/tops_tees-07.png"
      height: 2000
      width: 2000
    }
    color_info {
      colors: "RED"
    }
    sizes: "XXL"
  }
  variant_rollup_values {
    key: "inventory(store789,price)"
    value {
      list_value {
        values {
          number_value: 1.25
        }
      }
    }
  }
}
results {
  id: "16720"
  product {
    name: "pro

### Navigate in Query Results next page

In [170]:
#Query Results
from  google.cloud.retail_v2.services.search_service.pagers import SearchPager
search_request.page_token=response.next_page_token
response= client.search(search_request)
#next_page_results = SearchPager(response, search_request, response  )
print(response)

SearchPager<results {
  id: "18482"
  product {
    name: "projects/486742359899/locations/global/catalogs/default_catalog/branches/1/products/18482"
    categories: "Men > Active > Active"
    title: "ExOfficio Men\'s BugsAway Halo Long Sleeve Shirt"
    brands: "ExOfficio"
    attributes {
      key: "estimatedDeliveryDay"
      value {
        numbers: 4
        searchable: false
        indexable: true
      }
    }
    price_info {
      currency_code: "USD"
      price: 99
      original_price: 99
      cost: 41.382
    }
    uri: "https://retail-demo.mystorefront.org/product/18482"
    images {
      uri: "https://storage.googleapis.com/css-retail-images/subcat_active.png"
      height: 200
      width: 146
    }
    color_info {
      colors: "RED"
    }
    sizes: "XXL"
  }
}
results {
  id: "16753"
  product {
    name: "projects/486742359899/locations/global/catalogs/default_catalog/branches/1/products/16753"
    categories: "Men > Clothing > Tops & Tees"
    title: "Play La

## Last but not least, let's call a product in our catalog directly

In [174]:
client.product_path('486742359899','global','default_catalog',1,16684)

'projects/486742359899/locations/global/catalogs/default_catalog/branches/1/products/16684'

In [175]:
from google.cloud.retail_v2.services.product_service import ProductServiceClient
project,credentials = google.auth.default(quota_project_id='pod-fr-retail')
client = ProductServiceClient()
#client.update_product('projects/486742359899/locations/global/catalogs/default_catalog/branches/1/products/16684')
product= client.get_product(name=client.product_path('486742359899','global','default_catalog',1,16684))

In [176]:
print(product)

name: "projects/486742359899/locations/global/catalogs/default_catalog/branches/1/products/16684"
id: "16684"
type_: PRIMARY
primary_product_id: "16684"
categories: "Men > Clothing > Tops & Tees"
title: "Diesel Men\'s Stombol-S Shirt"
brands: "Diesel"
language_code: "en-US"
attributes {
  key: "estimatedDeliveryDay"
  value {
    numbers: 4
    searchable: false
    indexable: true
  }
}
price_info {
  currency_code: "USD"
  price: 98
  original_price: 98
  cost: 56.644
  price_range {
  }
}
availability: IN_STOCK
available_quantity {
  value: 10
}
fulfillment_info {
  type_: "custom-type-1"
  place_ids: "10"
}
uri: "https://retail-demo.mystorefront.org/product/16684"
images {
  uri: "https://storage.googleapis.com/css-retail-images/tops_tees-07.png"
  height: 2000
  width: 2000
}
audience {
  genders: "male"
}
color_info {
  colors: "RED"
}
sizes: "XXL"
conditions: "new"
retrievable_fields {
  paths: "title"
  paths: "categories"
  paths: "brands"
  paths: "attributes.estimated_delive